# Heros Of Pymoli Data Analysis:
- Male players purchased more than four times more than female and non-specified gender players combined.
- 19 -22 year olds purchased the most
- Players in the 39 - 42 year old range spent approximately 1.5 times the amount that players in the 19 - 22 year range spent.
- Item price did not negatively impact purchasing decision. In fact, the highest priced item (Orenmir, $4.95), was one of the most profitable items.

In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "resources/purchase_data.json"
df = pd.read_json(file)
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


##### Total Number of Players

In [3]:
#total number of players
# this should be unique count of SN (Screen Name)
players = pd.DataFrame({"Unique Player Count":[df["SN"].nunique()]})
players

,Unique Player Count
0,573


##### Purchasing Analysis (Total)

In [4]:
# Number of Unique Items
itemCount = df["Item ID"].nunique()

# Average Purchase Price
avgPrice = df["Price"].mean()
avgPrice = "${:.2f}".format(avgPrice)

# Total Number of Purchases
purchaseCount = df["Item ID"].count()

# Total Revenue
totalRevenue = df["Price"].sum()
totalRevenue = "${:,.2f}".format(totalRevenue)

# let's put all these in a new table
analysis = pd.DataFrame(
    {"Number of Unique Items":[itemCount], 
     "Average Purchase Price":[avgPrice],
     "Total Number of Purchases":[purchaseCount],
     "Total Revenue":[totalRevenue]}
)
analysis

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,$2.93,183,780,"$2,286.33"


##### Gender Demographics

In [5]:
gender = df.groupby("Gender")["SN"].nunique().to_frame("Unique Player Count")
gender["Percentage"] = (gender["Unique Player Count"]/gender["Unique Player Count"].sum()) * 100
gender["Percentage"] = gender["Percentage"].apply("{:.2f}%".format)
gender

,Unique Player Count,Percentage
Gender,,
Female,100,17.45%
Male,465,81.15%
Other / Non-Disclosed,8,1.40%


##### Purchasing Analysis (Gender)

In [6]:
# Purchase Count
genderPurchaseCount = df.groupby("Gender")["Item ID"].count().to_frame("Purchase Count")
# Average Purchase Price
genderAveragePurchasePrice = df.groupby("Gender")["Price"].mean().to_frame("Average Purchase Price")
# Total Purchase Value
genderTotalPurchaseAmount = df.groupby("Gender")["Price"].sum().to_frame("Total Purchase Amount")
# Normalized Totals
genderUniquePlayers = df.groupby("Gender")["SN"].nunique().to_frame("Player Count")
# join the tpa (total purchase amount) and the up (unique players) for additional manipulation
genderNormalizedAverage = genderTotalPurchaseAmount.merge(genderUniquePlayers, left_index=True, right_index=True)
genderNormalizedAverage["Normalized Average"] = genderNormalizedAverage["Total Purchase Amount"]/genderNormalizedAverage["Player Count"] 

# merge all the data
genderMerged = genderPurchaseCount.merge(genderAveragePurchasePrice, left_index=True, right_index=True).merge(genderNormalizedAverage, left_index=True, right_index=True) 

# format the dollar amounts
genderMerged["Average Purchase Price"] = genderMerged["Average Purchase Price"].apply("${:.2f}".format)
genderMerged["Total Purchase Amount"] = genderMerged["Total Purchase Amount"].apply("${:.2f}".format)
genderMerged["Normalized Average"] = genderMerged["Normalized Average"].apply("${:.2f}".format)
genderMerged

,Purchase Count,Average Purchase Price,Total Purchase Amount,Player Count,Normalized Average
Gender,,,,,
Female,136,$2.82,$382.91,100,$3.83
Male,633,$2.95,$1867.68,465,$4.02
Other / Non-Disclosed,11,$3.25,$35.74,8,$4.47


##### Purchasing Analysis (Age)

In [7]:
# create a range to use as bins of age groups of 4
# add 4 to max to ensure we get highest age bracket/bin
ageBins = np.arange(df.Age.min(),df["Age"].max()+4,4)
ageBinLabels = []
binLength = len(str(ageBins.max()))
mask = binLength * "0"

for index, a in enumerate(ageBins):
    if index != len(ageBins)-1:
        ageBinLabels.append((mask + str(a))[-binLength:] + " - " + str((int(a+3))))
df["AgeBin"] = pd.cut(df["Age"],ageBins, right=False, include_lowest=True)
df["AgeBinLabel"] = pd.cut(df["Age"],ageBins, right=False, include_lowest=True, labels=ageBinLabels)

# Purchase Count
agePurchaseCount = df.groupby("AgeBinLabel")["Item ID"].count().to_frame("Purchase Count")

# Average Purchase Price
ageAveragePurchasePrice = df.groupby("AgeBinLabel")["Price"].mean().to_frame("Average Purchase Price")

# Total Purchase Value
ageTotalPurchaseAmount = df.groupby("AgeBinLabel")["Price"].sum().to_frame("Total Purchase Amount")

# Normalized Totals
#get the unique number of purchasers for each age bin
ageUniquePlayers = df.groupby("AgeBinLabel")["SN"].nunique().to_frame("Player Count")

ageUniquePlayers["Percentage of Players"] = (ageUniquePlayers["Player Count"]/ageUniquePlayers["Player Count"].sum()) * 100


#join the total purchase amount and unique players to get the normalized average price
ageNormalizedAverage = ageTotalPurchaseAmount.merge(ageUniquePlayers, left_index=True, right_index=True)
#add a column and populate normalized average
ageNormalizedAverage["Normalized Average"] = ageNormalizedAverage["Total Purchase Amount"]/ageNormalizedAverage["Player Count"]

#merge all the data
ageMerged = agePurchaseCount.merge(ageAveragePurchasePrice, left_index=True, right_index=True).merge(ageNormalizedAverage, left_index=True, right_index=True)
#format the columns
ageMerged["Average Purchase Price"] = ageMerged["Average Purchase Price"].apply("${:.2f}".format)
ageMerged["Total Purchase Amount"] = ageMerged["Total Purchase Amount"].apply("${:.2f}".format)
ageMerged["Normalized Average"] = ageMerged["Normalized Average"].apply("${:.2f}".format) 
ageMerged["Percentage of Players"] = ageMerged["Percentage of Players"].apply("{:.2f}%".format)
ageMerged

,Purchase Count,Average Purchase Price,Total Purchase Amount,Player Count,Percentage of Players,Normalized Average
AgeBinLabel,,,,,,
07 - 10,32,$3.02,$96.62,22,3.84%,$4.39
11 - 14,31,$2.70,$83.79,20,3.49%,$4.19
15 - 18,111,$2.88,$319.32,84,14.66%,$3.80
19 - 22,231,$2.93,$676.20,178,31.06%,$3.80
23 - 26,207,$2.94,$608.02,153,26.70%,$3.97
27 - 30,63,$2.98,$187.99,44,7.68%,$4.27
31 - 34,46,$3.07,$141.24,34,5.93%,$4.15
35 - 38,37,$2.81,$104.06,25,4.36%,$4.16
39 - 42,20,$3.13,$62.56,11,1.92%,$5.69


##### Top Spenders

In [8]:
# Get the screen names of the 5 players who purchased the most
topSpenders = df.groupby("SN")["Price"].sum().nlargest(5).to_frame("Total Purchase Amount")
#merge withe full data set to see all purchases
topSpendersMerge = topSpenders.merge(df, left_index=True, right_on="SN")

# SN
# determined in initial query of top spenders
# Purchase Count
topSpendersPurchaseCount = topSpendersMerge.groupby("SN")["Item ID"].count().to_frame("Purchase Count")
# Average Purchase Price
topSpendersAveragePurchasePrice = topSpendersMerge.groupby("SN")["Price"].mean().to_frame("Average Purchase Price")
# Total Purchase Value
#calculated above to determine top spender

#merge data together
topSpendersAnalysis = topSpenders.merge(topSpendersPurchaseCount, left_index=True, right_index=True).merge(topSpendersAveragePurchasePrice, left_index=True, right_index=True)
#reorder the columns
cols = ["Purchase Count","Average Purchase Price","Total Purchase Amount"]
topSpendersAnalysis = topSpendersAnalysis[cols]
topSpendersAnalysis["Average Purchase Price"] = topSpendersAnalysis["Average Purchase Price"].map("${:.2f}".format)
topSpendersAnalysis["Total Purchase Amount"] = topSpendersAnalysis["Total Purchase Amount"].map("${:.2f}".format)
topSpendersAnalysis

,Purchase Count,Average Purchase Price,Total Purchase Amount
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


##### Most Popular Items

In [9]:
#Note: Different items have the same name - There was a 4-way tie for 2nd most popular item, so pulled 6 most popular items
mostPopular = df.groupby("Item ID")["Item ID"].count().nlargest(6).to_frame("Purchase Count")

# join to the raw data
mostPopularMerged = mostPopular.merge(df, left_index=True, right_on="Item ID")
#set the index to assist in removing duplicates
mostPopularIndexed = mostPopularMerged.set_index("Item ID")

# Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# we have these values
mostPopularIndexed.loc[:,["Item Name","Purchase Count","Price"]].drop_duplicates()

# Total Purchase Value - use the sum of the item from the df with all purchases
mostPopularTotalPurchaseAmount = mostPopularIndexed.groupby("Item ID")["Price"].sum().to_frame("Total Purchase Value")

#merge the most popular with the total purchase value data
mostPopularAnalysis = mostPopularIndexed.merge(mostPopularTotalPurchaseAmount, left_index=True, right_index=True)

# select only the columns we want to display and eliminate duplicates
mostPopularAnalysis = mostPopularAnalysis.loc[:,["Item Name","Purchase Count","Price","Total Purchase Value"]].drop_duplicates().sort_values(by=["Purchase Count"], ascending=False)
mostPopularAnalysis.columns=["Item Name","Purchase Count","Item Price","Total Purchase Value"]
mostPopularAnalysis["Item Price"] = mostPopularAnalysis["Item Price"].apply("${:.2f}".format)
mostPopularAnalysis["Total Purchase Value"] = mostPopularAnalysis["Total Purchase Value"].apply("${:.2f}".format)
mostPopularAnalysis

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
13,Serenity,9,$1.49,$13.41
31,Trickster,9,$2.07,$18.63
34,Retribution Axe,9,$4.14,$37.26
175,Woeful Adamantite Claymore,9,$1.24,$11.16


##### Most Profitable Items

In [10]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):

mostProfitable = df.groupby("Item ID")["Price"].sum().nlargest(5).to_frame("Total Purchase Value")
mostProfitableMerged = mostProfitable.merge(df, left_index=True, right_on="Item ID")
# set the index to help with deduping
mostProfitableIndexed = mostProfitableMerged.set_index("Item ID")
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value
mostProfitableIndexed["Purchase Count"] = mostProfitableIndexed.groupby("Item ID")["Item Name"].count()

mostProfitableIndexed = mostProfitableIndexed.loc[:,["Item Name","Purchase Count","Price","Total Purchase Value"]].drop_duplicates()
mostProfitableIndexed.columns = ["Item Name","Purchase Count","Item Price","Total Purchase Value"]
mostProfitableIndexed["Item Price"] = mostProfitableIndexed["Item Price"].apply("${:.2f}".format)
mostProfitableIndexed["Total Purchase Value"] = mostProfitableIndexed["Total Purchase Value"].apply("${:.2f}".format)
mostProfitableIndexed.sort_values(["Total Purchase Value"], ascending=False)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
